### Welcome to the Noise Stimulus Generator Notebook

# ToDO: time constant

The following parameters are taken as inputs:

- $T$ - the length of the stimulus in seconds
- $I_{0}$ - the mean input current in pA
- $\sigma $ - the standard deviation of the input current in pA
- $f$ - the sampling frequency in kHz

Input the parameters in the cell below and then choose `'Cell->Run All'` from the menu above to generate the file.

In [ ]:
# Parameters for stimulus file generation

T = 10 #length of stimulus required (s)

I_0 = 500 #mean input current (pA)

sigma = 100 #standard deviation (pA)

sampling_freq = 20000 # sampling frequency (kHz)

tau = 0.003 #time constant (s)

Export stimulation current to TPL file (patchmaster)
>Template Files for stimulation can be used in Voltage Clamp and in Current Clamp mode. In Current Clamp
mode, the stimulation data is defined in volts, where 1 mV corresponds with 1 pA current injection.
Example: A value of 0.1 would result in a current injection of 100 pA.

In [ ]:
%pylab inline

In [ ]:
from __future__ import division
import hashlib

In [ ]:
def OU_current(T, I_0,sigma, tau, sampling_freq=20000.):
    '''
    T : gives the simulation time in seconds
    other parameters as in Pozzorini et al 2015
    TODO: write docstring (params, etc)
    '''
    DeltaT = 1/sampling_freq
    
    def DeltaI(I):
        return (I_0-I)*DeltaT/tau + np.sqrt(2*sigma**2*DeltaT/tau)*randn()
    
    I = np.zeros(int(T/DeltaT))
    I[0] = I_0 #set the initial value to the mean input
    
    for i in xrange(1,len(I)):
        I[i] = I[i-1] + DeltaI(I[i-1])
        
    return I


def save_stim(T, I_0,sigma, tau, sampling_freq=20000.):

    I = OU_current(T, I_0,sigma, tau, sampling_freq=20000.) #in pA

    #get the command voltage () 
    I_mV = 1e-3*I            #convert the current in pA to mV
    I_mV = I_mV.astype('f4') #4 byte float format
    
    filename = 'I_noise_I0_{}pA_sigma_{}pA_T{}s.'.format(I_0,sigma,T)
    filename += hashlib.md5(I_mV).hexdigest()
    filename += '.tpl'
    
    # store the values in pA to a binary file
    I_mV.tofile(filename)
    
    print filename, 'File Saved!'
    
    return I

### Generate, save and view stimulus 

In [ ]:
I = save_stim(T,I_0,sigma,tau,sampling_freq)

time = linspace(0,T,sampling_freq*T)
plot(time,I)
xlabel('Time [s]')
ylabel('I [pA]')

title('Mean Current: {:.1f}pA Standard Deviaton: {:.1f}pA'.format(I.mean(), I.std()))